In [12]:
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
#Loader
srcRagFileName = "05 [참고] 2024년 블록체인 컨설팅 지원사업 수요기업 모집공고.pdf"


In [13]:
varRagDir="db11"
vectorstore = Chroma(persist_directory = varRagDir  , 
        embedding_function = 
        OpenAIEmbeddings(temperature=0

            )
        )
print(vectorstore._collection.count())

12


D:\myproject\lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! temperature is not default parameter.
                    temperature was transferred to model_kwargs.
                    Please confirm that temperature is what you intended.
  warnings.warn(


In [14]:
#  저장된 문서가 있는지 검색
result1 = vectorstore.get(where={"source": srcRagFileName})  # AI바우쳐 관련 내용
print("삭제전 컬럼수 : " , len(result1))
print("삭제전 리스트 건수: ", len(result1['ids']) )

삭제전 컬럼수 :  4
삭제전 리스트 건수:  6


In [15]:
for i in range(0, len(result1['ids']) ):
    # print(i)
    # print(result1['documents'][i])
    print(result1['ids'][i])

f659c82d-dc78-11ee-9472-30894a3f67b7
f659c82e-dc78-11ee-8465-30894a3f67b7
f659c82f-dc78-11ee-9db1-30894a3f67b7
f659c830-dc78-11ee-a6e8-30894a3f67b7
f659c831-dc78-11ee-93e1-30894a3f67b7
f659c832-dc78-11ee-b1ce-30894a3f67b7


In [16]:
# 데이터 삭제 
# https://python.langchain.com/docs/integrations/vectorstores/chroma#update-and-delete
print("데이터 삭제 시작!!!!")
for i in range(0, len(result1['ids']) ):
    print(i)
    iNum = result1['ids'][i]
    vectorstore._collection.delete(ids=iNum)

데이터 삭제 시작!!!!
0
1
2
3
4
5


In [17]:
result2 = vectorstore.get(where={"source": srcRagFileName})  # AI바우쳐 관련 내용
print("삭제후 컬럼수 : " , len(result2))
print("삭제후 리스트 건수: ", len(result2['ids']) )

삭제후 컬럼수 :  4
삭제후 리스트 건수:  0


In [18]:
# 대상문서 RAG에 저장 
loader = PyPDFLoader(srcRagFileName)
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

varRagDBDir="db11"
vectorstore = Chroma.from_documents \
    (documents=all_splits, 
         embedding = 
         OpenAIEmbeddings(temperature=0
                         
                         ), 
           persist_directory=varRagDBDir
    )

retriever = vectorstore.as_retriever()



In [19]:
#  저장된 문서가 있는지 검색
result3 = vectorstore.get(where={"source": srcRagFileName})  # AI바우쳐 관련 내용
print("저장후 컬럼수 : " , len(result3))
print("저장후 리스트 건수: ", len(result3['ids']) )

저장후 컬럼수 :  4
저장후 리스트 건수:  6


In [20]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", 
                 temperature=0, 
                
                )

template = """ 
학습된 문서내에서 존대말로 답변해 주세요 학습된 문서내의 질문이 아니면 '질문하신 내용은 업무와 관련이 없습니다. '라고 대답해 주세요
{context}
Question: {question}
Answer:"""

prompt = PromptTemplate.from_template(template)

qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
)

In [22]:
question="AI바우처가 뭐예요?"
result = qa_chain.invoke(question).content   
print(result)

AI바우처는 AI를 적용하고자 하는 수요기업에게 최적의 AI를 도입할 수 있도록 지원하고, 인공지능 솔루션을 개발한 공급기업에게는 새로운 시장 창출의 기회를 제공하는 사업의 일환으로, 정부가 수요기업에게 최대 2억원의 바우처를 지급하여 AI솔루션을 구매할 수 있도록 지원하는 프로그램입니다.


In [21]:
question="블록체인 컨설팅 지원사업이 뭐예요?"
result = qa_chain.invoke(question).content   
print(result)

블록체인 컨설팅 지원사업은 산업 전반에서 블록체인 기술을 도입하고자 하는 기업이나 기관, 블록체인 기술을 활용하여 비즈니스를 하는 기업, 그리고 글로벌 시장에 진출하고자 하는 기업에 맞춤형 컨설팅을 제공하는 사업입니다. 이를 통해 블록체인 기술의 안정적인 성장과 글로벌 시장 진출을 지원하고자 합니다.
